In [1]:
import numpy as np
import sys
import os
from os import path
import argparse
import math

caffe_root = '../../'  # this file is expected to be in {caffe_root}/examples
sys.path.insert(0, caffe_root + 'python')
import caffe

orgmodel = 'bvlc_googlenet.caffemodel'
orgnet = 'deploy.prototxt'

dstmodel = 'bvlc_googlenet_bp.caffemodel'
dstnet = 'deploy_bp.prototxt'

orgmodel2 = 'bvlc_googlenet_copy.caffemodel'


if True:
    
    
    # Load the original network and extract the fully connected layers' parameters.
    
    net = caffe.Net(orgnet, orgmodel, caffe.TEST)
    net.save(orgmodel2)
    params = net.params.keys()[-1:]
    print 'layers that will be converted: {}'.format(params)
    
    fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}
    #print 'FC params', shape(fc_params)

    for fc in params:
        print '{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape)

    # Load the fully convolutional network to transplant the parameters.
    net_full_conv = caffe.Net(dstnet, orgmodel, caffe.TEST)
    params_full_conv = net_full_conv.params.keys()[-1:]
    
    conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}
 
    for conv in params_full_conv:
        print '{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape)

    for pr, pr_conv in zip(params, params_full_conv):
        conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
        conv_params[pr_conv][1][...] = fc_params[pr][1]

    #print 'FC params', shape(conv_params)        
    net_full_conv.save(dstmodel)
    
    


layers that will be converted: ['loss3/classifier']
loss3/classifier weights are (1000, 1024) dimensional and biases are (1000,) dimensional
loss3/classifier_bp weights are (1000, 1024, 1, 1) dimensional and biases are (1000,) dimensional
